In [1]:
# Data structs, format and parsing
import json
from nested_lookup import nested_lookup, get_all_keys

# Graph Data
from py2neo import Graph, Node, Relationship, RelationshipMatch

# Others
from tqdm import tqdm

from beagle.transformers import SysmonTransformer

In [2]:
outputDir = ".\\sample_dataset\\output\\"
outputfile_json = outputDir + "preprocessed.json"

with open(outputfile_json) as f:
    jsondata = json.load(f)


In [3]:
"""Extract nested values from a JSON tree."""
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [4]:
jsondata[0]

{'Event': {'#attributes': {'xmlns': 'http://schemas.microsoft.com/win/2004/08/events/event'},
  'EventData': {'Status': '0x0',
   'SubjectDomainName': 'HQCORP',
   'SubjectLogonId': '0x2f336f',
   'SubjectUserName': 'administrator',
   'SubjectUserSid': 'S-1-5-21-1913345275-1711810662-261465553-500',
   'Workstation': '2016DC'},
  'System': {'Channel': 'Security',
   'Computer': '2016dc.hqcorp.local',
   'Correlation': {'#attributes': {'ActivityID': '3B48C871-DFE6-0000-A5C8-483BE6DFD201'}},
   'EventID': 4794,
   'EventRecordID': 3139859,
   'Execution': {'#attributes': {'ProcessID': 792, 'ThreadID': 1648}},
   'Keywords': '0x8020000000000000',
   'Level': 0,
   'Opcode': 0,
   'Provider': {'#attributes': {'Guid': '54849625-5478-4994-A5BA-3E3B0328C30D',
     'Name': 'Microsoft-Windows-Security-Auditing'}},
   'Security': None,
   'Task': 13824,
   'TimeCreated': {'#attributes': {'SystemTime': '2017-06-09T19:21:26.968669Z'}},
   'Version': 0}}}

# Add to neo4j graph DB

In [5]:
# set up authentication parameters
graph = Graph(password="admin")

In [44]:
# Graph dumps
graph.delete_all()

In [6]:
def tuplepair(n1_n2_r, dictlist):
    triplesList = []
    n1 = n1_n2_r[0]
    n2 = n1_n2_r[1]
    r = n1_n2_r[2]
    for ind, records in enumerate(dictlist):
        node1 = json_extract(records, n1)
        node2 = json_extract(records, n2)
        
        if node1 and node2:
            triplesList.append({
                "node1type" : n1,
                "node1label" : str(node1[0]), #to convert to str so can query in str easily and not have to use int in cypher
                "node2type" : n2,
                "node2label": str(node2[0]),
                "relationship": r,
                "logindex" : ind
            })
    return triplesList

def addRelationship(tripleList):
    for dic in tripleList:
        tx = graph.begin()
        node1 = Node(dic["node1type"], label = dic["node1label"], index = dic["logindex"])
        node2 = Node(dic["node2type"], label = dic["node2label"])
        newRelationship = Relationship(node1, dic["relationship"], node2)
        
        tx.merge(newRelationship, dic["node1type"], "label")
        tx.commit()

In [7]:
for items in tqdm(availablekeys):
    n1_n2_r = ("Computer",items, "has")
    tuplelist = tuplepair(n1_n2_r, jsondata)
    addRelationship(tuplelist)

NameError: name 'availablekeys' is not defined

In [10]:
tx = graph.begin()
node1 = Node("host", name = "PC123")
node2 = Node("Software", name = "Avenger")
newRelationship = Relationship(node1, "start_process" , node2)

tx.mer(newRelationship, "host", "name")
tx.commit()

AttributeError: 'Transaction' object has no attribute 'merge_one'